In [112]:
import numpy as np
import math
from copy import deepcopy
from typing import List, Callable

In [113]:
def bin_to_float(bitstring: str) -> float:

    if len(bitstring) != 16 or not all(digit in '01' for digit in bitstring):
        raise ValueError("Invalid bitstring length")
    
    int_val = int(bitstring, 2)
    bytes_val = int_val.to_bytes(2, byteorder='big')

    return np.frombuffer(bytes_val, dtype='f2')[0]
    
print(bin_to_float('0000000000000001'))

1.526e-05


In [114]:
class Qubit():

    superposition_amplitude = 1 / np.sqrt(2)

    def __init__(self, measurement: bool) -> None:

        self.measurement: bool = measurement
        self.alpha: float = self.superposition_amplitude
        self.beta: float = self.superposition_amplitude
        self.superposition: bool = False
        
    def apply_hadamard(self) -> 'Qubit':
        superposition_amplitude = 1 / np.sqrt(2)

        self.alpha = superposition_amplitude
        self.beta = superposition_amplitude
        self.superposition = True
        
        return self
        
    def apply_xgate(self) -> None:
        self.measurement = not self.measurement
        
    def measure(self) -> bool:
        if not self.superposition:
            return self.measurement
        
        random_var = np.random.uniform(0, 1)
        self.measurement = np.pow(self.beta, 2) >= random_var
        self.superposition = False
        
        return self.measurement
    
    def __str__(self) -> str:
        return f'Qubit[superposition: {self.superposition}, [measurement: {self.measurement}]]'

def single_qubit_measurement(qubit: Qubit) -> bool:
    return qubit.measure()

def full_measurement(quantum_chromosome: List[Qubit]) -> str:

    for gene in quantum_chromosome:
        gene.measure()
        
    return ''.join(map(lambda x: str(int(x.measure())), quantum_chromosome))

def crossover(
        queen: List[Qubit],
        male: List[Qubit],
        num_swap_qubits: int
    ) -> str:

    quantum_chromosome = deepcopy(queen)
    random_ind = np.random.randint(0, len(queen) - num_swap_qubits)
    
    for i in range(random_ind, random_ind + num_swap_qubits):

        if queen[i].measure() == male[i].measure():
            quantum_chromosome[i].apply_hadamard()

        else:
            quantum_chromosome[i].apply_xgate()
    
    return full_measurement(quantum_chromosome)

In [ ]:
def evolution(
        population_size: int,
        fitness: Callable[[float], float], #TODO make it pass multidimension fitness functions
        qubits_in_indiv: int = 16,
        num_males: int = 20,
        num_elites: int = 20,
        max_iteration: int = 500,
        crossover_size: int = 3
    ) -> float:
        
    population = []

    for i in range(population_size):
        quantum_chromosome = [Qubit(False).apply_hadamard() for _ in range(qubits_in_indiv)]
        quantum_chromosome_measured = [str(int(qubit.measure())) for qubit in quantum_chromosome]
        new_chromosome = ''.join(quantum_chromosome_measured)
        population.append(new_chromosome)
        
    population.sort(key = lambda x: fitness(bin_to_float(x))) # descending order if maximize 
    
    for indiv in population:
        print(bin_to_float(indiv))
    
    queen = population[0]
    males = population[1: num_males]
    remaining = population[num_males:]
    # elite = population[:num_elites]
    for _ in range(max_iteration):
        for i in range(population_size - num_elites, population_size):
            population[i] = crossover(
                    [Qubit(bool(digit)) for digit in queen],
                    [Qubit(bool(digit)) for digit in males[np.random.randint(0, num_males - 1)]],
                    crossover_size
                )
            
        population.sort(key = lambda x: fitness(bin_to_float(x))) # descending order if maximize 

        queen = population[0]
        males = population[1:num_males]
        
        print(f'Iteration: {_}, Queen: {bin_to_float(queen)}, Fitness: {fitness(bin_to_float(queen))}')

    return bin_to_float(queen)

In [116]:
def fitness_first(x):
    return (x - 15) ** 2 

def fitness_second(x):
    if not -5 <= x <= 5:
        return float('inf')  # Penalize out-of-bounds
    
    A = 10
    rastrigin = A + (x**2 - A * math.cos(2 * math.pi * x))
    discontinuity = 50 if -0.1 < x < 0.1 else 0
    noise = np.random.uniform(-0.5, 0.5)
    
    return rastrigin + discontinuity + noise

def deceptive_function(x):

    if not -10 <= x <= 10:
        return float('inf')  # Penalize out-of-bounds
    
    schwefel = 418.9829 - x * math.sin(math.sqrt(abs(x)))
    
    plateau = 100 if -2 <= x <= 2 else 0
    dip = -50 if -0.1 < x < 0.1 else 0
    poly = 0.1 * x**4 - 2 * x**2
    
    return schwefel + plateau + dip + poly

In [117]:
evolution(1000, fitness=deceptive_function)

3.352
3.094
3.535
2.82
2.688
3.785
3.873
2.396
2.379
2.355
2.236
-2.904
-2.912
-2.816
-3.395
-2.623
-2.613
-3.457
-2.459
-2.363
-2.27
-2.154
-3.744
-2.025
-3.838
-3.943
4.633
4.64
-4.32
4.92
5.07
5.12
-5.06
-5.113
5.42
-5.27
5.496
-5.336
5.656
5.832
5.914
5.93
5.55e-05
-0.0182
0.03008
-0.005768
0.07544
7.36e-05
-9.346e-05
0.003117
-0.03683
0.003483
0.0514
0.0869
-0.001314
-0.001444
0.002872
-0.0004325
-0.0002378
-0.001076
-0.0007377
-0.07855
0.000977
-1.39e-05
0.005333
0.002108
-0.014114
-0.05957
-0.0215
-0.0003726
0.01317
0.007233
0.0007443
0.0405
-0.003017
-0.002989
0.043
0.00864
0.0001231
-0.02611
-0.05014
0.00193
-0.08795
-0.001038
0.05222
0.003202
0.0003893
0.013176
-0.00010526
0.0315
0.00102
-0.03903
0.001655
-0.0005183
0.02914
-0.003775
0.01643
-0.01465
0.02042
6.56e-05
-0.0056
-0.003439
0.01533
-0.006237
0.0002997
0.00204
-0.0001531
-0.005955
-0.0783
0.001469
-0.002127
0.0131
0.06046
0.001681
-0.000632
-0.00555
-3.54e-05
-2.897e-05
-0.001731
-0.000641
0.0001819
0.01029
0.001465

np.float16(3.352)